In [ ]:
# | default_exp _cli.db

In [ ]:
from airt._testing import activate_by_import

[INFO] airt._testing.activate_by_import: Testing environment activated.


In [ ]:
# | export

from typing import *

In [ ]:
# | exporti

import datetime as dt
import json
import os

import pandas as pd
import typer
from tabulate import tabulate
from typer import echo

from airt._cli import helper
from airt._constant import CLIENT_DB_PASSWORD, CLIENT_DB_USERNAME
from airt._logger import get_logger, set_level
from airt.client import Client

In [ ]:
import logging
import shutil
import tempfile
from contextlib import contextmanager
from pathlib import Path
from urllib.parse import quote_plus as urlquote

import pytest
from azure.identity import DefaultAzureCredential
from azure.mgmt.storage import StorageManagementClient
from sqlmodel import create_engine
from typer.testing import CliRunner

import airt._sanitizer
from airt._constant import (
    CLIENT_DB_PASSWORD,
    CLIENT_DB_USERNAME,
    CLIENT_NAME,
    SERVER_URL,
    SERVICE_PASSWORD,
    SERVICE_TOKEN,
    SERVICE_USERNAME,
)
from airt.client import DataBlob

In [ ]:
# | exporti

app = typer.Typer(
    help="""A set of commands for importing and processing data from sources such as CSV/parquet files, databases, AWS S3 buckets, and Azure Blob Storage."""
)

In [ ]:
runner = CliRunner()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
set_level(logging.WARNING)

In [ ]:
# Testing logger settings

display(logger.getEffectiveLevel())
assert logger.getEffectiveLevel() == logging.WARNING

logger.debug("This is a debug message")
logger.info("This is an info")
logger.warning("This is a warning")
logger.error("This is an error")

30

[WARNING] __main__: This is a warning
[ERROR] __main__: This is an error


In [ ]:
# Helper context manager for testing

_airt_service_token = None


@contextmanager
def set_airt_service_token_envvar():
    global _airt_service_token
    if _airt_service_token is None:
        display("_airt_service_token is None, getting a token...")

        username = os.environ[SERVICE_USERNAME]
        password = os.environ[SERVICE_PASSWORD]

        Client.get_token(username=username, password=password)
        _airt_service_token = Client.auth_token

    try:
        os.environ[SERVICE_TOKEN] = _airt_service_token

        yield
    finally:
        del os.environ[SERVICE_TOKEN]

In [ ]:
with set_airt_service_token_envvar():
    display("*" * len((os.environ[SERVICE_TOKEN])))

'_airt_service_token is None, getting a token...'

'*******************************************************************************************************************************'

In [ ]:
def assert_has_help(xs: List[str]):
    result = runner.invoke(app, xs + ["--help"])

    display(result.stdout)
    assert " ".join(xs) in result.stdout

In [ ]:
TEST_S3_URI = "s3://test-airt-service/ecommerce_behavior_notebooks"
TEST_S3_CSV_URI = "s3://test-airt-service/ecommerce_behavior_csv"
TEST_AZURE_URI = "https://testairtservice.blob.core.windows.net/test-container/ecommerce_behavior_notebooks"
RANDOM_UUID_FOR_TESTING = "00000000-0000-0000-0000-000000000000"

In [ ]:
def remove_hypens_from_id(id: str) -> str:
    return "".join((id).split("-"))

In [ ]:
actual = remove_hypens_from_id(RANDOM_UUID_FOR_TESTING)
assert len(actual) == 32
actual

'00000000000000000000000000000000'

In [ ]:
# helper function to create a datablob


_db = None


@contextmanager
def generate_db(force_create: bool = False):
    global _db

    if _db is None or force_create:
        _db = DataBlob.from_s3(
            uri=TEST_S3_URI,
            access_key=os.environ["AWS_ACCESS_KEY_ID"],
            secret_key=os.environ["AWS_SECRET_ACCESS_KEY"],
        )

        display(f"{_db.uuid=}")
        assert len(remove_hypens_from_id(_db.uuid)) == 32

        _db.progress_bar()

    yield _db

In [ ]:
# | exporti


@app.command()
@helper.display_formated_table
@helper.requires_auth_token
def details(
    uuid: str = typer.Argument(
        ...,
        help="Datablob uuid.",
    ),
    format: Optional[str] = typer.Option(
        None,
        "--format",
        "-f",
        help="Format output and show only the given column(s) values.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> Dict["str", Union[pd.DataFrame, str]]:
    """Return details of a datablob."""

    from airt.client import DataBlob

    db = DataBlob(uuid=uuid)
    df = db.details()

    df["pulled_on"] = helper.humanize_date(df["pulled_on"])
    df["folder_size"] = helper.humanize_size(df["folder_size"])

    return {"df": df}

In [ ]:
# | include: false

assert_has_help(["details"])

'Usage: details [OPTIONS] UUID\n\n  Return details of a datablob.\n\nArguments:\n  UUID  Datablob uuid.  [required]\n\nOptions:\n  -f, --format TEXT     Format output and show only the given column(s) values.\n  -d, --debug           Set logger level to DEBUG and output everything.\n  --install-completion  Install completion for the current shell.\n  --show-completion     Show completion for the current shell, to copy it or\n                        customize the installation.\n  --help                Show this message and exit.\n'

In [ ]:
# Tests for details
# Testing positive scenario

# Helper function to extract ID


def extract_id(res) -> str:
    r = (res.split("\n")[1]).strip()
    return r.split(" ")[0]


with set_airt_service_token_envvar():
    with generate_db() as db:
        db_uuid = db.uuid

        # Getting Details of the data source
        format_str = "{'datablob_uuid': '{}'}"
        result = runner.invoke(app, [db_uuid, "--format", format_str])

        display(result.stdout)

        assert result.exit_code == 0
        assert result.stdout == f"{db_uuid}\n", f"{result.stdout=} {db_uuid=}"

"_db.uuid='823c12cd-1c46-4136-96f3-d22841a447c9'"

100%|██████████| 1/1 [00:15<00:00, 15.24s/it]


'823c12cd-1c46-4136-96f3-d22841a447c9\n'

In [ ]:
# | include: false

# Tests for details
# Testing negative scenario. Passing invalid data_id

with set_airt_service_token_envvar():
    data_uuid = RANDOM_UUID_FOR_TESTING
    result = runner.invoke(app, [data_uuid])

    display(result.stdout)

    assert result.exit_code == 1

'Error: The datablob uuid is incorrect. Please try again.\n'

In [ ]:
# | exporti


@app.command("from-s3")
@helper.requires_auth_token
def from_s3(
    uri: str = typer.Argument(..., help="The AWS S3 bucket uri."),
    access_key: Optional[str] = typer.Option(
        None,
        help="Access key for the S3 bucket. If **None** (default value), then the value from **AWS_ACCESS_KEY_ID** environment variable is used.",
    ),
    secret_key: Optional[str] = typer.Option(
        None,
        help="Secret key for the S3 bucket. If **None** (default value), then the value from **AWS_SECRET_ACCESS_KEY** environment variable is used.",
    ),
    cloud_provider: Optional[str] = typer.Option(
        None,
        "--cloud-provider",
        "-cp",
        help="The destination cloud storage provider's name to store the datablob. Currently, the API only supports **aws** and **azure** as cloud storage providers. If **None** (default value), then **aws**  will be used as the cloud storage provider.",
    ),
    region: Optional[str] = typer.Option(
        None,
        "--region",
        "-r",
        help="The destination cloud provider's region to save your datablob. If **None** (default value) then the default region will be assigned based on the cloud provider. "
        "In the case of **aws**, the datablob's source bucket region will be used and in the case of **azure**, **westeurope** will be used. "
        "The supported AWS regions are: ap-northeast-1, ap-northeast-2, ap-south-1, ap-southeast-1, ap-southeast-2, ca-central-1, eu-central-1, "
        "eu-north-1, eu-west-1, eu-west-2, eu-west-3, sa-east-1, us-east-1, us-east-2, us-west-1, us-west-2. The supported Azure Blob Storage "
        "regions are: australiacentral, australiacentral2, australiaeast, australiasoutheast, brazilsouth, canadacentral, canadaeast, centralindia, "
        "centralus, eastasia, eastus, eastus2, francecentral, francesouth, germanynorth, germanywestcentral, japaneast, japanwest, koreacentral, koreasouth, "
        "northcentralus, northeurope, norwayeast, norwaywest, southafricanorth, southafricawest, southcentralus, southeastasia, southindia, switzerlandnorth, "
        "switzerlandwest, uaecentral, uaenorth, uksouth, ukwest, westcentralus, westeurope, westindia, westus, westus2.",
    ),
    tag: Optional[str] = typer.Option(
        None,
        "--tag",
        "-t",
        help="A string to tag the datablob. If not passed, then the tag **latest** will be assigned to the datablob.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output datablob uuid only.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
):
    """Create and return a datablob that encapsulates the data from an AWS S3 bucket."""

    from airt.client import DataBlob

    db = DataBlob.from_s3(
        uri=uri,
        access_key=access_key,
        secret_key=secret_key,
        cloud_provider=cloud_provider,
        region=region,
        tag=tag,
    )

    if quiet:
        db.wait()

        typer.echo(f"{db.uuid}")
    else:
        typer.echo(f"Pulling datablob uuid: {db.uuid}")

        db.progress_bar()

In [ ]:
# | include: false

assert_has_help(["from-s3"])

"Usage: root from-s3 [OPTIONS] URI\n\n  Create and return a datablob that encapsulates the data from an AWS S3 bucket.\n\nArguments:\n  URI  The AWS S3 bucket uri.  [required]\n\nOptions:\n  --access-key TEXT           Access key for the S3 bucket. If **None** (default\n                              value), then the value from **AWS_ACCESS_KEY_ID**\n                              environment variable is used.\n  --secret-key TEXT           Secret key for the S3 bucket. If **None** (default\n                              value), then the value from\n                              **AWS_SECRET_ACCESS_KEY** environment variable is\n                              used.\n  -cp, --cloud-provider TEXT  The destination cloud storage provider's name to\n                              store the datablob. Currently, the API only\n                              supports **aws** and **azure** as cloud storage\n                              providers. If **None** (default value), then\n                  

In [ ]:
# Helper function to test multiple scenarios.


def assert_datablob(xs: List[str]):
    # Testing Negative scenario
    # Creating datablob without token

    # Clearing previously set env variables
    _token_flag = False

    if os.environ.get(SERVICE_TOKEN):
        _token_flag = True
        airt_service_token = os.environ[SERVICE_TOKEN]
        del os.environ[SERVICE_TOKEN]

    result = runner.invoke(app, xs)
    display(result.stdout)
    assert result.exit_code == 1
    assert (
        f"KeyError: The environment variable '{SERVICE_TOKEN}' is not set.\n\nPlease run the command '{CLIENT_NAME} token'"
        in result.stdout
    )

    if _token_flag:
        os.environ[SERVICE_TOKEN] = airt_service_token

    # Testing Positive scenario
    # With and without quite

    with set_airt_service_token_envvar():
        # Without quiet (verbose)
        result = runner.invoke(app, xs)
        display(result.stdout)
        assert "Pulling datablob uuid:" in result.stdout, result.stdout

        # With quiet
        display("*" * 120)
        result = runner.invoke(app, xs + ["-q"])
        display(result.stdout)
        assert len(remove_hypens_from_id(result.stdout[:-1])) == 32, len(
            result.stdout[:-1]
        )

In [ ]:
# Tests for Datablob s3

cmd = ["from-s3", f"{TEST_S3_CSV_URI}"]

assert_datablob(cmd)

"KeyError: The environment variable 'AIRT_SERVICE_TOKEN' is not set.\n\nPlease run the command 'airt token' to get the application token and set it in the environment variable `AIRT_SERVICE_TOKEN`.\n\nTry 'airt token --help' for help.\n"

'Pulling datablob uuid: f48bdad3-cbdf-4afc-9d35-38c626a58a2e\n\r  0%|          | 0/1 [00:00<?, ?it/s]\r  0%|          | 0/1 [00:05<?, ?it/s]\r  0%|          | 0/1 [00:10<?, ?it/s]\r  0%|          | 0/1 [00:15<?, ?it/s]\r  0%|          | 0/1 [00:20<?, ?it/s]\r  0%|          | 0/1 [00:25<?, ?it/s]\r  0%|          | 0/1 [00:30<?, ?it/s]\r  0%|          | 0/1 [00:35<?, ?it/s]\r100%|██████████| 1/1 [00:40<00:00,  5.06s/it]\r100%|██████████| 1/1 [00:40<00:00, 40.54s/it]\n'

'************************************************************************************************************************'

'6969f7a0-8a18-49a2-af7f-8d7b9f639c08\n'

In [ ]:
# | exporti


@app.command("from-azure-blob-storage")
@helper.requires_auth_token
def from_azure_blob_storage(
    uri: str = typer.Argument(..., help="Azure Blob Storage URI of the source file."),
    credential: str = typer.Option(
        ...,
        "--credential",
        "-c",
        help="Credential to access the Azure Blob Storage.",
    ),
    cloud_provider: Optional[str] = typer.Option(
        None,
        "--cloud-provider",
        "-cp",
        help="The destination cloud storage provider's name to store the datablob. Currently, the API only supports **aws** and **azure** as cloud storage providers. If **None** (default value), then **azure**  will be used as the cloud storage provider.",
    ),
    region: Optional[str] = typer.Option(
        None,
        "--region",
        "-r",
        help="The destination cloud provider's region to save your datablob. If **None** (default value) then the default region will be assigned based on the cloud provider. "
        "In the case of **aws**, **eu-west-1** will be used and in the case of **azure**, **westeurope** will be used. "
        "The supported AWS regions are: ap-northeast-1, ap-northeast-2, ap-south-1, ap-southeast-1, ap-southeast-2, ca-central-1, eu-central-1, "
        "eu-north-1, eu-west-1, eu-west-2, eu-west-3, sa-east-1, us-east-1, us-east-2, us-west-1, us-west-2. The supported Azure Blob Storage "
        "regions are: australiacentral, australiacentral2, australiaeast, australiasoutheast, brazilsouth, canadacentral, canadaeast, centralindia, "
        "centralus, eastasia, eastus, eastus2, francecentral, francesouth, germanynorth, germanywestcentral, japaneast, japanwest, koreacentral, koreasouth, "
        "northcentralus, northeurope, norwayeast, norwaywest, southafricanorth, southafricawest, southcentralus, southeastasia, southindia, switzerlandnorth, "
        "switzerlandwest, uaecentral, uaenorth, uksouth, ukwest, westcentralus, westeurope, westindia, westus, westus2.",
    ),
    tag: Optional[str] = typer.Option(
        None,
        "--tag",
        "-t",
        help="A string to tag the datablob. If not passed, then the tag **latest** will be assigned to the datablob.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output datablob uuid only.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
):
    """Create and return a datablob that encapsulates the data from an Azure Blob Storage."""

    from airt.client import DataBlob

    db = DataBlob.from_azure_blob_storage(
        uri=uri,
        credential=credential,
        cloud_provider=cloud_provider,
        region=region,
        tag=tag,
    )

    if quiet:
        db.wait()

        typer.echo(f"{db.uuid}")
    else:
        typer.echo(f"Pulling datablob uuid: {db.uuid}")

        db.progress_bar()

In [ ]:
assert_has_help(["from-azure-blob-storage"])

"Usage: root from-azure-blob-storage [OPTIONS] URI\n\n  Create and return a datablob that encapsulates the data from an Azure Blob\n  Storage.\n\nArguments:\n  URI  Azure Blob Storage URI of the source file.  [required]\n\nOptions:\n  -c, --credential TEXT       Credential to access the Azure Blob Storage.\n                              [required]\n  -cp, --cloud-provider TEXT  The destination cloud storage provider's name to\n                              store the datablob. Currently, the API only\n                              supports **aws** and **azure** as cloud storage\n                              providers. If **None** (default value), then\n                              **azure**  will be used as the cloud storage\n                              provider.\n  -r, --region TEXT           The destination cloud provider's region to save\n                              your datablob. If **None** (default value) then\n                              the default region will be assigne

In [ ]:
# Tests for from-azure-blob-storage
# Positive Scenario: Passing credential in arguments

storage_client = StorageManagementClient(
    DefaultAzureCredential(), os.environ["AZURE_SUBSCRIPTION_ID"]
)
keys = storage_client.storage_accounts.list_keys("test-airt-service", "testairtservice")
credential = keys.keys[0].value

cmd = [
    "from-azure-blob-storage",
    f"{TEST_AZURE_URI}",
    "--credential",
    f"{credential}",
]

assert_datablob(cmd)

"KeyError: The environment variable 'AIRT_SERVICE_TOKEN' is not set.\n\nPlease run the command 'airt token' to get the application token and set it in the environment variable `AIRT_SERVICE_TOKEN`.\n\nTry 'airt token --help' for help.\n"

'Pulling datablob uuid: 82e15303-adcd-4f56-9347-0fb3dbf8b9d2\n\r  0%|          | 0/1 [00:00<?, ?it/s]\r  0%|          | 0/1 [00:05<?, ?it/s]\r  0%|          | 0/1 [00:10<?, ?it/s]\r  0%|          | 0/1 [00:15<?, ?it/s]\r  0%|          | 0/1 [00:20<?, ?it/s]\r  0%|          | 0/1 [00:25<?, ?it/s]\r  0%|          | 0/1 [00:30<?, ?it/s]\r100%|██████████| 1/1 [00:35<00:00,  5.06s/it]\r100%|██████████| 1/1 [00:35<00:00, 35.50s/it]\n'

'************************************************************************************************************************'

'eb7d485d-6a9b-499a-9157-262077c1cbef\n'

In [ ]:
# Tests for from-azure-blob-storage
# Positive Scenario: Validating the default region
with set_airt_service_token_envvar():
    for region in ["westeurope", "northeurope"]:
        cmd = [
            "from-azure-blob-storage",
            f"{TEST_AZURE_URI}",
            "--credential",
            f"{credential}",
            "--cloud-provider",
            "azure",
            "--region",
            f"{region}",
            "-q",
        ]

        result = runner.invoke(app, cmd)
        display(result.stdout)

        db_uuid = result.stdout[:-1]
        display(db_uuid)
        assert len(remove_hypens_from_id(db_uuid)) == 32

        result = runner.invoke(app, ["details", db_uuid])
        display(result.stdout)
        assert result.exit_code == 0
        assert region in result.stdout

'7acc7398-1d8e-4454-8ded-e355f8414c7a\n'

'7acc7398-1d8e-4454-8ded-e355f8414c7a'

'datablob_uuid                         datasource_uuids    type                source                                                                                     region      cloud_provider    tags    pulled_on      folder_size    user_uuid                             error    disabled    ready\n7acc7398-1d8e-4454-8ded-e355f8414c7a  <none>              azure_blob_storage  https://testairtservice.blob.core.windows.net/test-container/ecommerce_behavior_notebooks  westeurope  azure             latest  7 seconds ago  10.2 MB        4b5131a3-6562-413d-abf2-1103275bf945  <none>   False       True\n'

'7c376650-5d91-4124-9057-9dfdd6984fbb\n'

'7c376650-5d91-4124-9057-9dfdd6984fbb'

'datablob_uuid                         datasource_uuids    type                source                                                                                     region       cloud_provider    tags    pulled_on       folder_size    user_uuid                             error    disabled    ready\n7c376650-5d91-4124-9057-9dfdd6984fbb  <none>              azure_blob_storage  https://testairtservice.blob.core.windows.net/test-container/ecommerce_behavior_notebooks  northeurope  azure             latest  11 seconds ago  10.2 MB        4b5131a3-6562-413d-abf2-1103275bf945  <none>   False       True\n'

In [ ]:
# | exporti


@app.command("from-mysql")
@helper.requires_auth_token
def from_mysql(
    host: str = typer.Option(..., help="Remote database host name."),
    database: str = typer.Option(..., help="Database name."),
    table: str = typer.Option(..., help="Table name."),
    port: int = typer.Option(
        3306,
        help="Host port number. If not passed, then the default value **3306** will be used.",
    ),
    cloud_provider: Optional[str] = typer.Option(
        None,
        "--cloud-provider",
        "-cp",
        help="The destination cloud storage provider's name to store the datablob. Currently, the API only supports **aws** and **azure** as cloud storage providers. If **None** (default value), then **aws**  will be used as the cloud storage provider.",
    ),
    region: Optional[str] = typer.Option(
        None,
        "--region",
        "-r",
        help="The destination cloud provider's region to save your datablob. If **None** (default value) then the default region will be assigned based on the cloud provider. "
        "In the case of **aws**, **eu-west-1** will be used and in the case of **azure**, **westeurope** will be used. "
        "The supported AWS regions are: ap-northeast-1, ap-northeast-2, ap-south-1, ap-southeast-1, ap-southeast-2, ca-central-1, eu-central-1, "
        "eu-north-1, eu-west-1, eu-west-2, eu-west-3, sa-east-1, us-east-1, us-east-2, us-west-1, us-west-2. The supported Azure Blob Storage "
        "regions are: australiacentral, australiacentral2, australiaeast, australiasoutheast, brazilsouth, canadacentral, canadaeast, centralindia, "
        "centralus, eastasia, eastus, eastus2, francecentral, francesouth, germanynorth, germanywestcentral, japaneast, japanwest, koreacentral, koreasouth, "
        "northcentralus, northeurope, norwayeast, norwaywest, southafricanorth, southafricawest, southcentralus, southeastasia, southindia, switzerlandnorth, "
        "switzerlandwest, uaecentral, uaenorth, uksouth, ukwest, westcentralus, westeurope, westindia, westus, westus2.",
    ),
    username: Optional[str] = typer.Option(
        None,
        "--username",
        "-u",
        help=f'Database username. If not passed, the default value "root" will be used unless the value is explicitly set in the environment variable **{CLIENT_DB_USERNAME}**.',
    ),
    password: Optional[str] = typer.Option(
        None,
        "--password",
        "-p",
        help=f'Database password. If not passed, the default value "" will be used unless the value is explicitly set in the environment variable **{CLIENT_DB_PASSWORD}**.',
    ),
    tag: Optional[str] = typer.Option(
        None,
        "--tag",
        "-t",
        help="A string to tag the datablob. If not passed, then the tag **latest** will be assigned to the datablob.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output datablob uuid only.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
):
    """Create and return a datablob that encapsulates the data from a mysql database.

    If the database requires authentication, pass the username/password as commandline arguments or store it in
    the **AIRT_CLIENT_DB_USERNAME** and **AIRT_CLIENT_DB_PASSWORD** environment variables.
    """

    from airt.client import DataBlob

    db = DataBlob.from_mysql(
        host=host,
        database=database,
        port=port,
        table=table,
        username=username,
        password=password,
        cloud_provider=cloud_provider,
        region=region,
        tag=tag,
    )

    if quiet:
        db.wait()
        typer.echo(f"{db.uuid}")
    else:
        typer.echo(f"Pulling datablob uuid: {db.uuid}")
        db.progress_bar()

In [ ]:
assert_has_help(["from-mysql"])

'Usage: root from-mysql [OPTIONS]\n\n  Create and return a datablob that encapsulates the data from a mysql database.\n\n  If the database requires authentication, pass the username/password as\n  commandline arguments or store it in the **AIRT_CLIENT_DB_USERNAME** and\n  **AIRT_CLIENT_DB_PASSWORD** environment variables.\n\nOptions:\n  --host TEXT                 Remote database host name.  [required]\n  --database TEXT             Database name.  [required]\n  --table TEXT                Table name.  [required]\n  --port INTEGER              Host port number. If not passed, then the default\n                              value **3306** will be used.  [default: 3306]\n  -cp, --cloud-provider TEXT  The destination cloud storage provider\'s name to\n                              store the datablob. Currently, the API only\n                              supports **aws** and **azure** as cloud storage\n                              providers. If **None** (default value), then\n           

In [ ]:
# tests for db. Testing negative scenario.
# Passing invalid host address

cmd = [
    "from-mysql",
    "--host",
    "db.staging.airt.ai",
    "--database",
    "test",
    "--table",
    "test",
]


with set_airt_service_token_envvar():
    # Without quiet (verbose)
    result = runner.invoke(app, cmd)
    display(result.stdout)
    assert "Unknown MySQL server host 'db.staging.airt.ai'" in result.stdout

'Pulling datablob uuid: 1a76ae7a-7668-4def-b172-0d50484f2530\n\r  0%|          | 0/1 [00:00<?, ?it/s]\r  0%|          | 0/1 [00:05<?, ?it/s]\nError: (MySQLdb.OperationalError) (2005, "Unknown MySQL server host \'db.staging.airt.ai\' (-2)")\n(Background on this error at: https://sqlalche.me/e/14/e3q8)\n'

In [ ]:
# tests for db. Testing positive scenario.

# Helper function to create new table in the mysql db


def get_db_engine():
    username = os.environ["DB_USERNAME"]
    password = os.environ["DB_PASSWORD"]
    host = os.environ["DB_HOST"]
    port = int(os.environ["DB_PORT"])
    database = os.environ["DB_DATABASE"]
    database_server = os.environ["DB_DATABASE_SERVER"]

    quoted_password = urlquote(password)
    conn_str = (
        f"{database_server}://{username}:{quoted_password}@{host}:{port}/{database}"
    )
    engine = create_engine(conn_str)

    return engine


with tempfile.TemporaryDirectory(prefix="test_s3_download_") as d:
    !aws s3 sync {TEST_S3_URI} {d}
    !ls {d}

    engine = get_db_engine()

    df = pd.read_parquet(d)
    try:
        df.to_sql("test_db_pull", con=engine, if_exists="fail")
    except ValueError as e:
        display(e)


with set_airt_service_token_envvar():
    for region in ["eu-west-1", "eu-west-3"]:
        # Creating a new datasource
        cmd = [
            "from-mysql",
            "--host",
            os.environ["DB_HOST"],
            "--database",
            os.environ["DB_DATABASE"],
            "--table",
            "test_db_pull",
            "--username",
            os.environ["DB_USERNAME"],
            "--password",
            os.environ["DB_PASSWORD"],
            "--tag",
            "v1.1.0",
        ]

        cmd_q = [
            "from-mysql",
            "--host",
            os.environ["DB_HOST"],
            "--database",
            os.environ["DB_DATABASE"],
            "--table",
            "test_db_pull",
            "--username",
            os.environ["DB_USERNAME"],
            "--password",
            os.environ["DB_PASSWORD"],
            "-cp",
            "aws",
            "--region",
            region,
            "-q",
        ]

        # Without quiet
        result = runner.invoke(app, cmd)

        display(result.stdout)
        assert "Pulling datablob uuid:" in str(result.stdout)

        # With quiet
        result = runner.invoke(app, cmd_q)
        db_uuid = result.stdout[:-1]
        display(db_uuid)
        assert len(remove_hypens_from_id(db_uuid)) == 32

        result = runner.invoke(app, ["details", db_uuid])
        display(result.stdout)
        assert result.exit_code == 0
        assert region in result.stdout

download: s3://test-airt-service/ecommerce_behavior_notebooks/_metadata to ../../../tmp/test_s3_download_s_7mrvfp/_metadata
download: s3://test-airt-service/ecommerce_behavior_notebooks/part.16.parquet to ../../../tmp/test_s3_download_s_7mrvfp/part.16.parquet
download: s3://test-airt-service/ecommerce_behavior_notebooks/part.17.parquet to ../../../tmp/test_s3_download_s_7mrvfp/part.17.parquet
download: s3://test-airt-service/ecommerce_behavior_notebooks/part.18.parquet to ../../../tmp/test_s3_download_s_7mrvfp/part.18.parquet
download: s3://test-airt-service/ecommerce_behavior_notebooks/part.19.parquet to ../../../tmp/test_s3_download_s_7mrvfp/part.19.parquet
download: s3://test-airt-service/ecommerce_behavior_notebooks/part.2.parquet to ../../../tmp/test_s3_download_s_7mrvfp/part.2.parquet
download: s3://test-airt-service/ecommerce_behavior_notebooks/part.4.parquet to ../../../tmp/test_s3_download_s_7mrvfp/part.4.parquet
download: s3://test-airt-service/ecommerce_behavior_notebooks/pa

'Pulling datablob uuid: d4b4b132-d5ef-4c41-ad31-0f2cc5fb5b02\n\r  0%|          | 0/1 [00:00<?, ?it/s]\r  0%|          | 0/1 [00:05<?, ?it/s]\r100%|██████████| 1/1 [00:10<00:00,  5.05s/it]\r100%|██████████| 1/1 [00:10<00:00, 10.15s/it]\n'

'79652c84-fdaf-445f-a9cd-95e4901e0eb4'

'datablob_uuid                         datasource_uuids    type    source                                               region     cloud_provider    tags    pulled_on      folder_size    user_uuid                             error    disabled    ready\n79652c84-fdaf-445f-a9cd-95e4901e0eb4  <none>              db      mysql://harish-mysql:3306/airt_service/test_db_pull  eu-west-1  aws               latest  3 seconds ago  8.0 MB         4b5131a3-6562-413d-abf2-1103275bf945  <none>   False       True\n'

'Pulling datablob uuid: 155a1e9c-2e52-4639-ad26-b462dc0315c4\n\r  0%|          | 0/1 [00:00<?, ?it/s]\r  0%|          | 0/1 [00:00<?, ?it/s]\r  0%|          | 0/1 [00:05<?, ?it/s]\r  0%|          | 0/1 [00:10<?, ?it/s]\r100%|██████████| 1/1 [00:15<00:00,  5.06s/it]\r100%|██████████| 1/1 [00:15<00:00, 15.50s/it]\n'

'868229e8-0f95-4eb5-8db6-30e329654a0d'

'datablob_uuid                         datasource_uuids    type    source                                               region     cloud_provider    tags    pulled_on      folder_size    user_uuid                             error    disabled    ready\n868229e8-0f95-4eb5-8db6-30e329654a0d  <none>              db      mysql://harish-mysql:3306/airt_service/test_db_pull  eu-west-3  aws               latest  3 seconds ago  8.0 MB         4b5131a3-6562-413d-abf2-1103275bf945  <none>   False       True\n'

In [ ]:
# | exporti


@app.command("to-datasource")
@helper.requires_auth_token
def to_datasource(
    uuid: str = typer.Option(
        ...,
        help="Datablob uuid.",
    ),
    file_type: str = typer.Option(
        ...,
        help='The file type of the datablob. Currently, the API only supports "csv" and "parquet" as file types.',
    ),
    index_column: str = typer.Option(
        ...,
        help="The column to use as index (row labels).",
    ),
    sort_by: str = typer.Option(
        ...,
        help="The column(s) to sort the data. Can either be a string or a JSON encoded list of strings.",
    ),
    deduplicate_data: bool = typer.Option(
        False,
        help="If set to **True** (default value **False**), the datasource will be created with duplicate rows removed.",
    ),
    blocksize: str = typer.Option(
        "256MB",
        help="The number of bytes used to split larger files. If None, then the default value **256MB** will be used.",
    ),
    kwargs_json: Optional[str] = typer.Option(
        None,
        help="Additional JSON encoded dict arguments to use while processing the data.e.g: To skip 100 lines from the bottom of file, pass '{"
        '"skipfooter"'
        ": 100}'",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output datasource uuid only.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
):
    """Process the datablob and return a datasource object."""

    from airt.client import DataBlob

    kwargs = json.loads(kwargs_json) if kwargs_json else {}

    try:
        sort_by = json.loads(sort_by)

    except json.JSONDecodeError as e:
        pass

    db = DataBlob(uuid=uuid)
    ds = db.to_datasource(
        file_type=file_type,
        index_column=index_column,
        sort_by=sort_by,
        deduplicate_data=deduplicate_data,
        blocksize=blocksize,
        **kwargs,
    )

    if quiet:
        ds.wait()
        typer.echo(f"{ds.uuid}")
    else:
        typer.echo(f"Processing and pulling the datasource uuid: {ds.uuid}")

        ds.progress_bar()

In [ ]:
assert_has_help(["to-datasource"])

'Usage: root to-datasource [OPTIONS]\n\n  Process the datablob and return a datasource object.\n\nOptions:\n  --uuid TEXT                     Datablob uuid.  [required]\n  --file-type TEXT                The file type of the datablob. Currently, the\n                                  API only supports "csv" and "parquet" as file\n                                  types.  [required]\n  --index-column TEXT             The column to use as index (row labels).\n                                  [required]\n  --sort-by TEXT                  The column(s) to sort the data. Can either be\n                                  a string or a JSON encoded list of strings.\n                                  [required]\n  --deduplicate-data / --no-deduplicate-data\n                                  If set to **True** (default value **False**),\n                                  the datasource will be created with duplicate\n                                  rows removed.  [default: no-deduplicate-data

In [ ]:
# Tests for to-datasource

with set_airt_service_token_envvar():
    with generate_db() as db:
        cmd = [
            "to-datasource",
            "--uuid",
            f"{db.uuid}",
            "--file-type",
            "parquet",
            "--index-column",
            "user_id",
            "--sort-by",
            "event_time",
            "--kwargs-json",
            '{"parse_dates": ["event_time"], "skipfooter": 100}',
        ]
        result = runner.invoke(app, cmd)

        display(result.stdout)
        assert result.exit_code == 0, f"{result.stdout=}, {result.exit_code=}"
        assert (
            "Processing and pulling the datasource uuid:" in result.stdout
        ), result.stdout

        cmd = [
            "to-datasource",
            "--uuid",
            f"{db.uuid}",
            "--file-type",
            "parquet",
            "--index-column",
            "user_id",
            "--sort-by",
            '["event_time", "category_id"]',
            "--kwargs-json",
            '{"parse_dates": ["event_time"], "skipfooter": 100}',
            "-q",
        ]
        result = runner.invoke(app, cmd)

        display(result.stdout)
        assert result.exit_code == 0, f"{result.stdout=}, {result.exit_code=}"
        assert len(result.stdout[:-1].replace("-", "").replace("\n", "")) == 32

'Processing and pulling the datasource uuid: 53bf8fc2-31c2-4b35-b355-5dcec1d79b4d\n\r  0%|          | 0/1 [00:00<?, ?it/s]\r  0%|          | 0/1 [00:05<?, ?it/s]\r  0%|          | 0/1 [00:10<?, ?it/s]\r  0%|          | 0/1 [00:15<?, ?it/s]\r  0%|          | 0/1 [00:20<?, ?it/s]\r  0%|          | 0/1 [00:25<?, ?it/s]\r100%|██████████| 1/1 [00:30<00:00,  5.06s/it]\r100%|██████████| 1/1 [00:30<00:00, 30.43s/it]\n'

'eb23a3bf-bf18-452e-a8db-1df7c52798b5\n'

In [ ]:
# Tests for to-datasource. Passing wrong index and sort column names

with set_airt_service_token_envvar():
    with generate_db() as db:
        cmd = [
            "to-datasource",
            "--uuid",
            f"{db.uuid}",
            "--file-type",
            "parquet",
            "--index-column",
            "random-col",
            "--sort-by",
            "random-col",
            "-q",
        ]
        result = runner.invoke(app, cmd)

        display(result.stdout)
        assert "'random-col'" in result.stdout, result.stdout

'Error: "Data has no column \'random-col\': use any column of [\'event_time\', \'event_type\', \'product_id\', \'category_id\', \'category_code\', \'brand\', \'price\', \'user_id\', \'user_session\']"\n'

In [ ]:
# | exporti


@app.command()
@helper.display_formated_table
@helper.requires_auth_token
def ls(
    offset: int = typer.Option(
        0,
        "--offset",
        "-o",
        help="The number of datablobs to offset at the beginning. If **None**, then the default value **0** will be used.",
    ),
    limit: int = typer.Option(
        100,
        "--limit",
        "-l",
        help="The maximum number of datablobs to return from the server. If **None**, then the default value **100** will be used.",
    ),
    disabled: bool = typer.Option(
        False,
        "--disabled",
        help="If set to **True**, then only the deleted datablobs will be returned."
        "Else, the default value **False** will be used to return only the list"
        "of active datablobs.",
    ),
    completed: bool = typer.Option(
        False,
        "--completed",
        help="If set to **True**, then only the datablobs that are successfully downloaded"
        "to the server will be returned. Else, the default value **False** will be used to"
        "return all the datablobs.",
    ),
    format: Optional[str] = typer.Option(
        None,
        "--format",
        "-f",
        help="Format output and show only the given column(s) values.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output only datablob uuids separated by space",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> Dict["str", Union[pd.DataFrame, str]]:
    """Return the list of datablobs."""

    from airt.client import DataBlob

    dbx = DataBlob.ls(
        offset=offset, limit=limit, disabled=disabled, completed=completed
    )

    df = DataBlob.as_df(dbx)

    df["pulled_on"] = helper.humanize_date(df["pulled_on"])
    df["folder_size"] = helper.humanize_size(df["folder_size"])

    return {"df": df, "quite_column_name": "datablob_uuid"}

In [ ]:
assert_has_help(["ls"])

'Usage: root ls [OPTIONS]\n\n  Return the list of datablobs.\n\nOptions:\n  -o, --offset INTEGER  The number of datablobs to offset at the beginning. If\n                        **None**, then the default value **0** will be used.\n                        [default: 0]\n  -l, --limit INTEGER   The maximum number of datablobs to return from the\n                        server. If **None**, then the default value **100** will\n                        be used.  [default: 100]\n  --disabled            If set to **True**, then only the deleted datablobs will\n                        be returned.Else, the default value **False** will be\n                        used to return only the listof active datablobs.\n  --completed           If set to **True**, then only the datablobs that are\n                        successfully downloadedto the server will be returned.\n                        Else, the default value **False** will be used toreturn\n                        all the datablobs.\n  -f

In [ ]:
# Tests for datasource_ls
# Testing positive scenario. Saving the token in env variable


def get_uuids_from_result(result) -> List[int]:
    return [uuid for uuid in result.stdout[:-1].split("\n")]


with set_airt_service_token_envvar():
    # Without quiet
    format_str = "{'datablob_uuid': '{}', 'type': '{}'}"
    result = runner.invoke(app, ["ls", "--format", format_str])
    display(result.stdout)

    assert "type" in result.stdout
    assert result.exit_code == 0

    # With quiet
    result = runner.invoke(app, ["ls", "-q"])
    display(result.stdout)

    assert result.exit_code == 0
    uuids = get_uuids_from_result(result)
    display(f"{uuids=}")

'datablob_uuid                         type\n8ef21b49-8e4d-402b-a64c-f70e6487eb49  s3\nb7c1fe3f-8187-4874-bd5b-f5d8363fc2fe  s3\n823c12cd-1c46-4136-96f3-d22841a447c9  s3\nf48bdad3-cbdf-4afc-9d35-38c626a58a2e  s3\n6969f7a0-8a18-49a2-af7f-8d7b9f639c08  s3\n82e15303-adcd-4f56-9347-0fb3dbf8b9d2  azure_blob_storage\neb7d485d-6a9b-499a-9157-262077c1cbef  azure_blob_storage\n7acc7398-1d8e-4454-8ded-e355f8414c7a  azure_blob_storage\n7c376650-5d91-4124-9057-9dfdd6984fbb  azure_blob_storage\n1a76ae7a-7668-4def-b172-0d50484f2530  db\nd4b4b132-d5ef-4c41-ad31-0f2cc5fb5b02  db\n79652c84-fdaf-445f-a9cd-95e4901e0eb4  db\n155a1e9c-2e52-4639-ad26-b462dc0315c4  db\n868229e8-0f95-4eb5-8db6-30e329654a0d  db\n'

'8ef21b49-8e4d-402b-a64c-f70e6487eb49\nb7c1fe3f-8187-4874-bd5b-f5d8363fc2fe\n823c12cd-1c46-4136-96f3-d22841a447c9\nf48bdad3-cbdf-4afc-9d35-38c626a58a2e\n6969f7a0-8a18-49a2-af7f-8d7b9f639c08\n82e15303-adcd-4f56-9347-0fb3dbf8b9d2\neb7d485d-6a9b-499a-9157-262077c1cbef\n7acc7398-1d8e-4454-8ded-e355f8414c7a\n7c376650-5d91-4124-9057-9dfdd6984fbb\n1a76ae7a-7668-4def-b172-0d50484f2530\nd4b4b132-d5ef-4c41-ad31-0f2cc5fb5b02\n79652c84-fdaf-445f-a9cd-95e4901e0eb4\n155a1e9c-2e52-4639-ad26-b462dc0315c4\n868229e8-0f95-4eb5-8db6-30e329654a0d\n'

"uuids=['8ef21b49-8e4d-402b-a64c-f70e6487eb49', 'b7c1fe3f-8187-4874-bd5b-f5d8363fc2fe', '823c12cd-1c46-4136-96f3-d22841a447c9', 'f48bdad3-cbdf-4afc-9d35-38c626a58a2e', '6969f7a0-8a18-49a2-af7f-8d7b9f639c08', '82e15303-adcd-4f56-9347-0fb3dbf8b9d2', 'eb7d485d-6a9b-499a-9157-262077c1cbef', '7acc7398-1d8e-4454-8ded-e355f8414c7a', '7c376650-5d91-4124-9057-9dfdd6984fbb', '1a76ae7a-7668-4def-b172-0d50484f2530', 'd4b4b132-d5ef-4c41-ad31-0f2cc5fb5b02', '79652c84-fdaf-445f-a9cd-95e4901e0eb4', '155a1e9c-2e52-4639-ad26-b462dc0315c4', '868229e8-0f95-4eb5-8db6-30e329654a0d']"

In [ ]:
# Tests for datasource_ls
# Testing positive scenario.
# Testing by passing different values for  limit


with set_airt_service_token_envvar():
    for limit in [1, 10, 1000]:
        offset = 1
        result = runner.invoke(app, ["ls", "--offset", offset, "--limit", limit, "-q"])

        assert result.exit_code == 0

        uuids = get_uuids_from_result(result)
        display(f"{uuids=}")
        assert limit >= len(uuids) >= 0

"uuids=['b7c1fe3f-8187-4874-bd5b-f5d8363fc2fe']"

"uuids=['b7c1fe3f-8187-4874-bd5b-f5d8363fc2fe', '823c12cd-1c46-4136-96f3-d22841a447c9', 'f48bdad3-cbdf-4afc-9d35-38c626a58a2e', '6969f7a0-8a18-49a2-af7f-8d7b9f639c08', '82e15303-adcd-4f56-9347-0fb3dbf8b9d2', 'eb7d485d-6a9b-499a-9157-262077c1cbef', '7acc7398-1d8e-4454-8ded-e355f8414c7a', '7c376650-5d91-4124-9057-9dfdd6984fbb', '1a76ae7a-7668-4def-b172-0d50484f2530', 'd4b4b132-d5ef-4c41-ad31-0f2cc5fb5b02']"

"uuids=['b7c1fe3f-8187-4874-bd5b-f5d8363fc2fe', '823c12cd-1c46-4136-96f3-d22841a447c9', 'f48bdad3-cbdf-4afc-9d35-38c626a58a2e', '6969f7a0-8a18-49a2-af7f-8d7b9f639c08', '82e15303-adcd-4f56-9347-0fb3dbf8b9d2', 'eb7d485d-6a9b-499a-9157-262077c1cbef', '7acc7398-1d8e-4454-8ded-e355f8414c7a', '7c376650-5d91-4124-9057-9dfdd6984fbb', '1a76ae7a-7668-4def-b172-0d50484f2530', 'd4b4b132-d5ef-4c41-ad31-0f2cc5fb5b02', '79652c84-fdaf-445f-a9cd-95e4901e0eb4', '155a1e9c-2e52-4639-ad26-b462dc0315c4', '868229e8-0f95-4eb5-8db6-30e329654a0d']"

In [ ]:
# Tests for datasource_ls
# Testing positive scenario.
# Testing by passing large value for offset.

with set_airt_service_token_envvar():
    limit = 10
    offset = 1_000_000
    result = runner.invoke(app, ["ls", "--offset", offset, "--limit", limit])

    assert result.exit_code == 0

    display(result.stdout)

'datablob_uuid    datasource_uuids    type    source    region    cloud_provider    tags    pulled_on    folder_size    ready\n'

In [ ]:
# | exporti


@app.command()
@helper.display_formated_table
@helper.requires_auth_token
def rm(
    uuid: str = typer.Argument(
        ...,
        help="Datablob uuid.",
    ),
    format: Optional[str] = typer.Option(
        None,
        "--format",
        "-f",
        help="Format output and show only the given column(s) values.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output the deleted datablob uuid only.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> Dict["str", Union[pd.DataFrame, str]]:
    """Delete a datablob from the server."""

    from airt.client import DataBlob

    db = DataBlob(uuid=uuid)
    df = db.delete()

    df["pulled_on"] = helper.humanize_date(df["pulled_on"])
    df["folder_size"] = helper.humanize_size(df["folder_size"])

    return {"df": df, "quite_column_name": "datablob_uuid"}

In [ ]:
assert_has_help(["rm"])

'Usage: root rm [OPTIONS] UUID\n\n  Delete a datablob from the server.\n\nArguments:\n  UUID  Datablob uuid.  [required]\n\nOptions:\n  -f, --format TEXT  Format output and show only the given column(s) values.\n  -q, --quiet        Output the deleted datablob uuid only.\n  -d, --debug        Set logger level to DEBUG and output everything.\n  --help             Show this message and exit.\n'

In [ ]:
# Tests for datasource rm
# Testing positive scenario with quite

with set_airt_service_token_envvar():
    with generate_db() as db:
        db_uuid = db.uuid

        # Deleting the created data source from the server
        result = runner.invoke(app, ["rm", db_uuid, "-q"])
        deleted_uuid = result.stdout[:-1]

        display(deleted_uuid)

        assert result.exit_code == 0
        assert deleted_uuid == db_uuid

        # List the existing data source ids in server and make sure the deleted id is not present in the server
        format_str = "{'datablob_uuid': '{}'}"
        ls_result = runner.invoke(app, ["ls", "--format", format_str])
        ls_uuids = get_uuids_from_result(ls_result)

        display(ls_uuids)
        assert deleted_uuid not in ls_uuids

        # ls with quiet and disabled = True
        result = runner.invoke(app, ["ls", "--disabled", "-q"])

        display(result.stdout)
        assert result.exit_code == 0

        disabled_db_uuids = get_uuids_from_result(result)

        display(f"{disabled_db_uuids=}")
        assert deleted_uuid in disabled_db_uuids

        # Testing negative scenario. Deleting already deleted data source
        result = runner.invoke(app, ["rm", deleted_uuid, "-q"])
        display(result.stdout)
        assert result.exit_code == 1

        # Testing negative scenario. Getting the details of the deleted data source
        result = runner.invoke(app, ["details", deleted_uuid])
        display(result.stdout)
        assert result.exit_code == 1

'823c12cd-1c46-4136-96f3-d22841a447c9'

['8ef21b49-8e4d-402b-a64c-f70e6487eb49',
 'b7c1fe3f-8187-4874-bd5b-f5d8363fc2fe',
 'f48bdad3-cbdf-4afc-9d35-38c626a58a2e',
 '6969f7a0-8a18-49a2-af7f-8d7b9f639c08',
 '82e15303-adcd-4f56-9347-0fb3dbf8b9d2',
 'eb7d485d-6a9b-499a-9157-262077c1cbef',
 '7acc7398-1d8e-4454-8ded-e355f8414c7a',
 '7c376650-5d91-4124-9057-9dfdd6984fbb',
 '1a76ae7a-7668-4def-b172-0d50484f2530',
 'd4b4b132-d5ef-4c41-ad31-0f2cc5fb5b02',
 '79652c84-fdaf-445f-a9cd-95e4901e0eb4',
 '155a1e9c-2e52-4639-ad26-b462dc0315c4',
 '868229e8-0f95-4eb5-8db6-30e329654a0d']

'823c12cd-1c46-4136-96f3-d22841a447c9\n'

"disabled_db_uuids=['823c12cd-1c46-4136-96f3-d22841a447c9']"

'Error: The datablob has already been deleted.\n'

'Error: The datablob has already been deleted.\n'

In [ ]:
# Tests for datasource rm
# Testing negative scenario. Deleting invalid data source
with set_airt_service_token_envvar():
    # Testing negative scenario. Deleting already deleted data source
    result = runner.invoke(app, ["rm", RANDOM_UUID_FOR_TESTING, "-q"])

    display(result.stdout)

'Error: The datablob uuid is incorrect. Please try again.\n'

In [ ]:
# | exporti


@app.command()
@helper.display_formated_table
@helper.requires_auth_token
def tag(
    uuid: str = typer.Option(
        ...,
        "--datablob_uuid",
        "-uuid",
        help="Datablob uuid in the server.",
    ),
    name: str = typer.Option(
        ...,
        "--name",
        "-n",
        help="A string to tag the datablob.",
    ),
    format: Optional[str] = typer.Option(
        None,
        "--format",
        "-f",
        help="Format output and show only the given column(s) values.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> Dict["str", Union[pd.DataFrame, str]]:
    """Tag an existing datablob in the server."""

    from airt.client import DataBlob

    db = DataBlob(uuid=uuid)
    df = db.tag(name=name)

    df["pulled_on"] = helper.humanize_date(df["pulled_on"])
    df["folder_size"] = helper.humanize_size(df["folder_size"])

    return {"df": df}

In [ ]:
assert_has_help(["tag"])

'Usage: root tag [OPTIONS]\n\n  Tag an existing datablob in the server.\n\nOptions:\n  -uuid, --datablob_uuid TEXT  Datablob uuid in the server.  [required]\n  -n, --name TEXT              A string to tag the datablob.  [required]\n  -f, --format TEXT            Format output and show only the given column(s)\n                               values.\n  -d, --debug                  Set logger level to DEBUG and output everything.\n  --help                       Show this message and exit.\n'

In [ ]:
# Tests for tag
# Testing positive scenario

with set_airt_service_token_envvar():
    with generate_db(force_create=True) as db:
        db_uuid = db.uuid

        # Tag the data source
        format_str = "{'datablob_uuid': '{}', 'tags': '{}'}"
        result = runner.invoke(
            app, ["tag", "-uuid", db_uuid, "-n", "v1.1.0", "--format", format_str]
        )

        display(result.stdout)

        assert result.exit_code == 0
        assert "v1.1.0" in str(result.stdout)

"_db.uuid='f2e00205-6568-4249-8f0e-176142cc5ac2'"

100%|██████████| 1/1 [00:10<00:00, 10.16s/it]


'datablob_uuid                         tags\nf2e00205-6568-4249-8f0e-176142cc5ac2  latest, v1.1.0\n'

In [ ]:
# | exporti


@app.command("from-local")
@helper.requires_auth_token
def from_local(
    path: str = typer.Option(
        ...,
        "--path",
        "-p",
        help="The relative or absolute path to a local CSV/parquet file or to a directory containing the CSV/parquet files.",
    ),
    cloud_provider: Optional[str] = typer.Option(
        None,
        "--cloud-provider",
        "-cp",
        help="The destination cloud storage provider's name to store the datablob. Currently, the API only supports **aws** and **azure** as cloud storage providers. If **None** (default value), then **aws**  will be used as the cloud storage provider.",
    ),
    region: Optional[str] = typer.Option(
        None,
        "--region",
        "-r",
        help="The destination cloud provider's region to save your datablob. If **None** (default value) then the default region will be assigned based on the cloud provider. "
        "In the case of **aws**, **eu-west-1** will be used and in the case of **azure**, **westeurope** will be used. "
        "The supported AWS regions are: ap-northeast-1, ap-northeast-2, ap-south-1, ap-southeast-1, ap-southeast-2, ca-central-1, eu-central-1, "
        "eu-north-1, eu-west-1, eu-west-2, eu-west-3, sa-east-1, us-east-1, us-east-2, us-west-1, us-west-2. The supported Azure Blob Storage "
        "regions are: australiacentral, australiacentral2, australiaeast, australiasoutheast, brazilsouth, canadacentral, canadaeast, centralindia, "
        "centralus, eastasia, eastus, eastus2, francecentral, francesouth, germanynorth, germanywestcentral, japaneast, japanwest, koreacentral, koreasouth, "
        "northcentralus, northeurope, norwayeast, norwaywest, southafricanorth, southafricawest, southcentralus, southeastasia, southindia, switzerlandnorth, "
        "switzerlandwest, uaecentral, uaenorth, uksouth, ukwest, westcentralus, westeurope, westindia, westus, westus2.",
    ),
    tag: Optional[str] = typer.Option(
        None,
        "--tag",
        "-t",
        help="A string to tag the datablob. If not passed, then the tag **latest** will be assigned to the datablob.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output data id only.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> None:
    """Create and return a datablob from local csv file.

    The API currently allows users to create datablobs from CSV or Parquet files. We intend to support additional file formats in future releases.
    """

    from airt.client import DataBlob

    if quiet:
        db = DataBlob.from_local(
            path=path,
            cloud_provider=cloud_provider,
            region=region,
            tag=tag,
            show_progress=False,
        )
        typer.echo(f"{db.uuid}")
    else:
        db = DataBlob.from_local(
            path=path, cloud_provider=cloud_provider, region=region, tag=tag
        )
        typer.echo(f"Successfully pulled the datablob uuid: {db.uuid}.")

In [ ]:
assert_has_help(["from-local"])

"Usage: root from-local [OPTIONS]\n\n  Create and return a datablob from local csv file.\n\n  The API currently allows users to create datablobs from CSV or Parquet files.\n  We intend to support additional file formats in future releases.\n\nOptions:\n  -p, --path TEXT             The relative or absolute path to a local\n                              CSV/parquet file or to a directory containing the\n                              CSV/parquet files.  [required]\n  -cp, --cloud-provider TEXT  The destination cloud storage provider's name to\n                              store the datablob. Currently, the API only\n                              supports **aws** and **azure** as cloud storage\n                              providers. If **None** (default value), then\n                              **aws**  will be used as the cloud storage\n                              provider.\n  -r, --region TEXT           The destination cloud provider's region to save\n                            

In [ ]:
# | include: false

# Helper function to download a sample csv file into the temp directory for testing Datablob local csv command


def get_test_csv_path() -> Path:
    """Downloads the account_312571_events from the s3 bucket and stores it in temp folder.
    Finally converts the downloaded account_312571_events files to a csv file and returns the
    path of the temp folder and the temp csv file.
    """
    temp_dirpath = Path(tempfile.mkdtemp(prefix="test_s3_download_"))

    !aws s3 sync {TEST_S3_URI} {temp_dirpath / "parquet"}

    parquet_path = Path(temp_dirpath / "parquet")
    csv_dirpath = Path(temp_dirpath / "csv")
    os.mkdir(csv_dirpath)

    for i, f in enumerate(list(parquet_path.glob("*.parquet"))):
        df = pd.read_parquet(f)
        df.to_csv(csv_dirpath / f"file-{i}.csv", index=False)

    display(list(csv_dirpath.glob("*")))

    return temp_dirpath, csv_dirpath


# Testing multiple files upload.

# Create temp directory
temp_dir, csv_dirpath = get_test_csv_path()

# Creating a new datasource
cmd = ["from-local", "--path", f"{csv_dirpath}"]

cmd_q = ["from-local", "--path", f"{csv_dirpath}", "-q"]

with set_airt_service_token_envvar():
    # Without quiet
    result = runner.invoke(app, cmd)

    display(result.stdout)
    assert "Successfully pulled the datablob uuid:" in str(result.stdout)

    # With quiet
    result = runner.invoke(app, cmd_q)
    db_uuid = result.stdout[:-1]
    display(db_uuid)
    assert len(remove_hypens_from_id(db_uuid)) == 32

    result = runner.invoke(app, ["details", db_uuid])
    display(result.stdout)
    assert result.exit_code == 0
    assert "eu-west-1" in result.stdout

# Deleting the temp directory
shutil.rmtree(temp_dir)
display(f"{temp_dir.exists()=}")
assert not temp_dir.exists()

download: s3://test-airt-service/ecommerce_behavior_notebooks/_common_metadata to ../../../tmp/test_s3_download_qoxtkeyn/parquet/_common_metadata
download: s3://test-airt-service/ecommerce_behavior_notebooks/part.16.parquet to ../../../tmp/test_s3_download_qoxtkeyn/parquet/part.16.parquet
download: s3://test-airt-service/ecommerce_behavior_notebooks/_metadata to ../../../tmp/test_s3_download_qoxtkeyn/parquet/_metadata
download: s3://test-airt-service/ecommerce_behavior_notebooks/part.17.parquet to ../../../tmp/test_s3_download_qoxtkeyn/parquet/part.17.parquet
download: s3://test-airt-service/ecommerce_behavior_notebooks/part.19.parquet to ../../../tmp/test_s3_download_qoxtkeyn/parquet/part.19.parquet
download: s3://test-airt-service/ecommerce_behavior_notebooks/part.18.parquet to ../../../tmp/test_s3_download_qoxtkeyn/parquet/part.18.parquet
download: s3://test-airt-service/ecommerce_behavior_notebooks/part.2.parquet to ../../../tmp/test_s3_download_qoxtkeyn/parquet/part.2.parquet
down

[Path('/tmp/test_s3_download_qoxtkeyn/csv/file-1.csv'),
 Path('/tmp/test_s3_download_qoxtkeyn/csv/file-15.csv'),
 Path('/tmp/test_s3_download_qoxtkeyn/csv/file-6.csv'),
 Path('/tmp/test_s3_download_qoxtkeyn/csv/file-8.csv'),
 Path('/tmp/test_s3_download_qoxtkeyn/csv/file-17.csv'),
 Path('/tmp/test_s3_download_qoxtkeyn/csv/file-19.csv'),
 Path('/tmp/test_s3_download_qoxtkeyn/csv/file-18.csv'),
 Path('/tmp/test_s3_download_qoxtkeyn/csv/file-5.csv'),
 Path('/tmp/test_s3_download_qoxtkeyn/csv/file-14.csv'),
 Path('/tmp/test_s3_download_qoxtkeyn/csv/file-0.csv'),
 Path('/tmp/test_s3_download_qoxtkeyn/csv/file-10.csv'),
 Path('/tmp/test_s3_download_qoxtkeyn/csv/file-7.csv'),
 Path('/tmp/test_s3_download_qoxtkeyn/csv/file-12.csv'),
 Path('/tmp/test_s3_download_qoxtkeyn/csv/file-4.csv'),
 Path('/tmp/test_s3_download_qoxtkeyn/csv/file-2.csv'),
 Path('/tmp/test_s3_download_qoxtkeyn/csv/file-3.csv'),
 Path('/tmp/test_s3_download_qoxtkeyn/csv/file-16.csv'),
 Path('/tmp/test_s3_download_qoxtkeyn/cs

'\r  0%|          | 0/20 [00:00<?, ?it/s]\r  5%|▌         | 1/20 [00:00<00:10,  1.85it/s]\r 10%|█         | 2/20 [00:01<00:10,  1.76it/s]\r 15%|█▌        | 3/20 [00:01<00:09,  1.77it/s]\r 20%|██        | 4/20 [00:02<00:08,  1.83it/s]\r 25%|██▌       | 5/20 [00:02<00:08,  1.73it/s]\r 30%|███       | 6/20 [00:03<00:07,  1.76it/s]\r 35%|███▌      | 7/20 [00:03<00:07,  1.72it/s]\r 40%|████      | 8/20 [00:04<00:06,  1.78it/s]\r 45%|████▌     | 9/20 [00:05<00:06,  1.78it/s]\r 50%|█████     | 10/20 [00:05<00:05,  1.80it/s]\r 55%|█████▌    | 11/20 [00:06<00:04,  1.85it/s]\r 60%|██████    | 12/20 [00:06<00:04,  1.80it/s]\r 65%|██████▌   | 13/20 [00:07<00:04,  1.57it/s]\r 70%|███████   | 14/20 [00:08<00:03,  1.64it/s]\r 75%|███████▌  | 15/20 [00:09<00:03,  1.39it/s]\r 80%|████████  | 16/20 [00:09<00:03,  1.32it/s]\r 85%|████████▌ | 17/20 [00:10<00:02,  1.28it/s]\r 90%|█████████ | 18/20 [00:11<00:01,  1.29it/s]\r 95%|█████████▌| 19/20 [00:12<00:00,  1.25it/s]\r100%|██████████| 20/20 [00:13<00:00

'e0015859-78a7-42dd-bd86-7db9f7f121a4'

'datablob_uuid                         datasource_uuids    type    source                                    region     cloud_provider    tags    pulled_on    folder_size    user_uuid                             error    disabled    ready\ne0015859-78a7-42dd-bd86-7db9f7f121a4  <none>              local   local:/tmp/test_s3_download_qoxtkeyn/csv  eu-west-1  aws               latest  None         unknown        4b5131a3-6562-413d-abf2-1103275bf945  <none>   False       False\n'

'temp_dir.exists()=False'

In [ ]:
# Testing single files upload.

# Create temp directory
temp_dir, csv_dirpath = get_test_csv_path()

# Creating a new datasource
cmd = ["from-local", "--path", str(csv_dirpath / "file-1.csv")]

cmd_q = ["from-local", "--path", str(csv_dirpath / "file-1.csv"), "-q"]


with set_airt_service_token_envvar():
    # Without quiet
    result = runner.invoke(app, cmd)

    display(result.stdout)
    assert "Successfully pulled the datablob uuid:" in str(result.stdout)

    # With quiet
    result = runner.invoke(app, cmd_q)

    display(result.stdout)
    assert len(remove_hypens_from_id(result.stdout[:-1])) == 32


# Deleting the temp directory
shutil.rmtree(temp_dir)
display(f"{temp_dir.exists()=}")
assert not temp_dir.exists()

download: s3://test-airt-service/ecommerce_behavior_notebooks/_metadata to ../../../tmp/test_s3_download_58lkcuny/parquet/_metadata
download: s3://test-airt-service/ecommerce_behavior_notebooks/_common_metadata to ../../../tmp/test_s3_download_58lkcuny/parquet/_common_metadata
download: s3://test-airt-service/ecommerce_behavior_notebooks/part.16.parquet to ../../../tmp/test_s3_download_58lkcuny/parquet/part.16.parquet
download: s3://test-airt-service/ecommerce_behavior_notebooks/part.12.parquet to ../../../tmp/test_s3_download_58lkcuny/parquet/part.12.parquet
download: s3://test-airt-service/ecommerce_behavior_notebooks/part.1.parquet to ../../../tmp/test_s3_download_58lkcuny/parquet/part.1.parquet
download: s3://test-airt-service/ecommerce_behavior_notebooks/part.11.parquet to ../../../tmp/test_s3_download_58lkcuny/parquet/part.11.parquet
download: s3://test-airt-service/ecommerce_behavior_notebooks/part.18.parquet to ../../../tmp/test_s3_download_58lkcuny/parquet/part.18.parquet
down

[Path('/tmp/test_s3_download_58lkcuny/csv/file-1.csv'),
 Path('/tmp/test_s3_download_58lkcuny/csv/file-15.csv'),
 Path('/tmp/test_s3_download_58lkcuny/csv/file-6.csv'),
 Path('/tmp/test_s3_download_58lkcuny/csv/file-8.csv'),
 Path('/tmp/test_s3_download_58lkcuny/csv/file-17.csv'),
 Path('/tmp/test_s3_download_58lkcuny/csv/file-19.csv'),
 Path('/tmp/test_s3_download_58lkcuny/csv/file-18.csv'),
 Path('/tmp/test_s3_download_58lkcuny/csv/file-5.csv'),
 Path('/tmp/test_s3_download_58lkcuny/csv/file-14.csv'),
 Path('/tmp/test_s3_download_58lkcuny/csv/file-0.csv'),
 Path('/tmp/test_s3_download_58lkcuny/csv/file-10.csv'),
 Path('/tmp/test_s3_download_58lkcuny/csv/file-7.csv'),
 Path('/tmp/test_s3_download_58lkcuny/csv/file-12.csv'),
 Path('/tmp/test_s3_download_58lkcuny/csv/file-4.csv'),
 Path('/tmp/test_s3_download_58lkcuny/csv/file-2.csv'),
 Path('/tmp/test_s3_download_58lkcuny/csv/file-3.csv'),
 Path('/tmp/test_s3_download_58lkcuny/csv/file-16.csv'),
 Path('/tmp/test_s3_download_58lkcuny/cs

'\r  0%|          | 0/1 [00:00<?, ?it/s]\r100%|██████████| 1/1 [00:00<00:00,  1.68it/s]\r100%|██████████| 1/1 [00:00<00:00,  1.68it/s]\nSuccessfully pulled the datablob uuid: 8feeb4d7-3343-4f1e-a25e-8f39822ee440.\n'

'bdfe2e41-a9da-4d68-9459-a03418eb7e0f\n'

'temp_dir.exists()=False'

In [ ]:
# | exporti


@app.command("from-clickhouse")
@helper.requires_auth_token
def from_clickhouse(
    host: str = typer.Option(..., help="Remote database host name."),
    database: str = typer.Option(..., help="Database name."),
    table: str = typer.Option(..., help="Table name."),
    protocol: str = typer.Option(
        ..., help='Protocol to use. The valid values are "native" and "http".'
    ),
    index_column: str = typer.Option(
        ..., help="The column to use as index (row labels)."
    ),
    timestamp_column: str = typer.Option(
        ..., help="Timestamp column name in the tabel."
    ),
    port: int = typer.Option(
        0,
        help="Host port number. If not passed, then the default value **0** will be used.",
    ),
    cloud_provider: Optional[str] = typer.Option(
        None,
        "--cloud-provider",
        "-cp",
        help="The destination cloud storage provider's name to store the datablob. Currently, the API only supports **aws** and **azure** as cloud storage providers. If **None** (default value), then **aws**  will be used as the cloud storage provider.",
    ),
    region: Optional[str] = typer.Option(
        None,
        "--region",
        "-r",
        help="The destination cloud provider's region to save your datablob. If **None** (default value) then the default region will be assigned based on the cloud provider. "
        "In the case of **aws**, **eu-west-1** will be used and in the case of **azure**, **westeurope** will be used. "
        "The supported AWS regions are: ap-northeast-1, ap-northeast-2, ap-south-1, ap-southeast-1, ap-southeast-2, ca-central-1, eu-central-1, "
        "eu-north-1, eu-west-1, eu-west-2, eu-west-3, sa-east-1, us-east-1, us-east-2, us-west-1, us-west-2. The supported Azure Blob Storage "
        "regions are: australiacentral, australiacentral2, australiaeast, australiasoutheast, brazilsouth, canadacentral, canadaeast, centralindia, "
        "centralus, eastasia, eastus, eastus2, francecentral, francesouth, germanynorth, germanywestcentral, japaneast, japanwest, koreacentral, koreasouth, "
        "northcentralus, northeurope, norwayeast, norwaywest, southafricanorth, southafricawest, southcentralus, southeastasia, southindia, switzerlandnorth, "
        "switzerlandwest, uaecentral, uaenorth, uksouth, ukwest, westcentralus, westeurope, westindia, westus, westus2.",
    ),
    username: Optional[str] = typer.Option(
        None,
        "--username",
        "-u",
        help="Database username. If not passed, the default value 'root' will be used unless the value is explicitly set in the environment variable **CLICKHOUSE_USERNAME**.",
    ),
    password: Optional[str] = typer.Option(
        None,
        "--password",
        "-p",
        help="Database password. If not passed, the default value '' will be used unless the value is explicitly set in the environment variable **CLICKHOUSE_PASSWORD**.",
    ),
    filters_json: Optional[str] = typer.Option(
        None,
        "--filters-json",
        "-f",
        help="Additional parameters to be used when importing data. For example, if you want to filter and extract data only for a specific user_id, pass '{"
        '"user_id"'
        ": 1}'.",
    ),
    tag: Optional[str] = typer.Option(
        None,
        "--tag",
        "-t",
        help="A string to tag the datablob. If not passed, then the tag **latest** will be assigned to the datablob.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output datablob uuid only.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
):
    """Create and return a datablob that encapsulates the data from a ClickHouse database.

    If the database requires authentication, pass the username/password as commandline arguments or store it in
    the **CLICKHOUSE_USERNAME** and **CLICKHOUSE_PASSWORD** environment variables.
    """

    filters = json.loads(filters_json) if filters_json else None

    from airt.client import DataBlob

    db = DataBlob.from_clickhouse(
        host=host,
        database=database,
        table=table,
        protocol=protocol,
        index_column=index_column,
        timestamp_column=timestamp_column,
        port=port,
        username=username,
        password=password,
        filters=filters,
        cloud_provider=cloud_provider,
        region=region,
        tag=tag,
    )

    if quiet:
        db.wait()
        typer.echo(f"{db.uuid}")
    else:
        typer.echo(f"Pulling datablob uuid: {db.uuid}")
        db.progress_bar()

In [ ]:
assert_has_help(["from-clickhouse"])

'Usage: root from-clickhouse [OPTIONS]\n\n  Create and return a datablob that encapsulates the data from a ClickHouse\n  database.\n\n  If the database requires authentication, pass the username/password as\n  commandline arguments or store it in the **CLICKHOUSE_USERNAME** and\n  **CLICKHOUSE_PASSWORD** environment variables.\n\nOptions:\n  --host TEXT                 Remote database host name.  [required]\n  --database TEXT             Database name.  [required]\n  --table TEXT                Table name.  [required]\n  --protocol TEXT             Protocol to use. The valid values are "native" and\n                              "http".  [required]\n  --index-column TEXT         The column to use as index (row labels).\n                              [required]\n  --timestamp-column TEXT     Timestamp column name in the tabel.  [required]\n  --port INTEGER              Host port number. If not passed, then the default\n                              value **0** will be used.  [default: 0

In [ ]:
# tests for from-clickhouse. Testing positive scenario.

cmd = [
    "from-clickhouse",
    "--host",
    os.environ.get("CLICKHOUSE_HOST"),
    "--database",
    os.environ.get("CLICKHOUSE_DATABASE"),
    "--table",
    os.environ.get("CLICKHOUSE_EVENTS_TABLE"),
    "--protocol",
    "native",
    "--index-column",
    "PersonId",
    "--timestamp-column",
    "OccurredTimeTicks",
    "--filters-json",
    '{"AccountId": 312571}',
]


with set_airt_service_token_envvar():
    # Without quiet (verbose)
    result = runner.invoke(app, cmd)
    display(result.stdout)
    assert "Pulling datablob uuid: " in result.stdout

    result = runner.invoke(app, cmd + ["-q"])
    db_uuid = result.stdout[:-1]
    display(db_uuid)
    assert len(remove_hypens_from_id(db_uuid)) == 32

    result = runner.invoke(app, ["details", db_uuid])
    display(result.stdout)
    assert result.exit_code == 0
    assert "eu-west-1" in result.stdout

    result = runner.invoke(app, cmd + ["-cp", "aws", "--region", "eu-west-3", "-q"])
    db_uuid = result.stdout[:-1]
    display(db_uuid)
    assert len(remove_hypens_from_id(db_uuid)) == 32

    result = runner.invoke(app, ["details", db_uuid])
    display(result.stdout)
    assert result.exit_code == 0
    assert "eu-west-3" in result.stdout

'Pulling datablob uuid: a17b8b8f-74a7-4d8e-9f53-803c14ad726f\n\r  0%|          | 0/1 [00:00<?, ?it/s]\r  0%|          | 0/1 [00:05<?, ?it/s]\r  0%|          | 0/1 [00:10<?, ?it/s]\r  0%|          | 0/1 [00:15<?, ?it/s]\r  0%|          | 0/1 [00:20<?, ?it/s]\r100%|██████████| 1/1 [00:25<00:00,  5.06s/it]\r100%|██████████| 1/1 [00:25<00:00, 25.40s/it]\n'

'b3826d4f-852b-4676-af05-933316c4a56a'

'datablob_uuid                         datasource_uuids    type    source                                                       region     cloud_provider    tags    pulled_on      folder_size    user_uuid                             error    disabled    ready\nb3826d4f-852b-4676-af05-933316c4a56a  <none>              db      clickhouse+native://20.73.247.24:0/infobip/airt_training_3m  eu-west-1  aws               latest  3 seconds ago  8.9 MB         4b5131a3-6562-413d-abf2-1103275bf945  <none>   False       True\n'

'247e6345-8940-4d75-8845-576d2b652955'

'datablob_uuid                         datasource_uuids    type    source                                                       region     cloud_provider    tags    pulled_on      folder_size    user_uuid                             error    disabled    ready\n247e6345-8940-4d75-8845-576d2b652955  <none>              db      clickhouse+native://20.73.247.24:0/infobip/airt_training_3m  eu-west-3  aws               latest  4 seconds ago  8.9 MB         4b5131a3-6562-413d-abf2-1103275bf945  <none>   False       True\n'

In [ ]:
# tests for from-clickhouse. Testing negative scenario.

cmd = [
    "from-clickhouse",
    "--host",
    os.environ.get("CLICKHOUSE_HOST"),
    "--database",
    "fake-database",
    "--table",
    "fake-table",
    "--protocol",
    "native",
    "--index-column",
    "PersonId",
    "--timestamp-column",
    "OccurredTimeTicks",
    "-f",
    '{"AccountId": 312571}',
]


with set_airt_service_token_envvar():
    # Without quiet (verbose)
    result = runner.invoke(app, cmd)
    display(result.stdout)
    assert result.exit_code == 1

"Pulling datablob uuid: b957e2c5-4527-41d8-90b2-46e0c1bada8f\n\r  0%|          | 0/1 [00:00<?, ?it/s]\r  0%|          | 0/1 [00:00<?, ?it/s]\r  0%|          | 0/1 [00:05<?, ?it/s]\r  0%|          | 0/1 [00:10<?, ?it/s]\nError: Orig exception: Code: 81.\nDB::Exception: Database `fake-database` doesn't exist. Stack trace:\n\n0. DB::Exception::Exception(DB::Exception::MessageMasked&&, int, bool) @ 0xddb0df5 in /usr/bin/clickhouse\n1. ? @ 0x883ade4 in /usr/bin/clickhouse\n2. DB::Databas\n"